# Hanlp-batch_ernie-gram

Chạy công cụ phân tích dữ liệu dựa vào ERNIE_GRAM, phân tích cả phần.
Chương trình này sẽ nhanh hơn `hanlp-single_ernie_gram``, nhưng có khả năng bị trành bộ nhớ
Hãy đổi sang `single_ernie_gram` nếu gặp vấn đề trên

## Bước 1: Cài đặt thư viện
Cần chạy trước khi chạy các bước khác

In [ ]:
%pip install hanlp requests

## Bước 2: Tải xuống dữ liệu + cài đặt chương trình
Tải xuống thư viện liên quan, bước này có thể sẽ hơi lâu

In [ ]:
import os, re, requests, getpass

URL = "https://chivi.app/_wn"
CLIENT = requests.Session()

os.environ["HANLP_HOME"] = ".hanlp" # đổi địa chỉ lưu models
import hanlp

# tải xuống dữ liệu 
MTL_TASK = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ERNIE_GRAM_ZH)

# đăng nhập vào hệ thống
def do_login():
  print("\nThông tin đăng nhập Chivi:")

  print("Hòm thư:", end=' ')
  email = input().strip()

  upass = getpass.getpass("Mật khẩu: ").strip()

  json = {'email': email, 'upass': upass}
  res = CLIENT.post("https://chivi.app/_db/_user/log-in", json=json)

  if res.status_code < 300:
    user = res.json()["uname"]
    print(f"Xin chào {user}!")
    print()
  else:
    print("Đăng nhập thất bại, mời thử lại!")
    do_login()

# lấy thông tin chương cuối cùng
def get_max_ch_no(wn_id, sname):
  url = f"{URL}/seeds/{wn_id}/{sname}/brief"
  res = CLIENT.get(url)
  return res.json()["chap_count"]

def render_con_data(con_data):
  output = ''

  for con_line in con_data:
    output += re.sub('\\n(\\s+)', ' ', str(con_line))
    output += '\n'

  return output

# phản hồi tới người dùng, bỏ qua phần đệm (chỉ chứa tên chương) mỗi chương
def print_msg(message, p_idx):
  if p_idx > 0:
    print(message % p_idx)

def analyze_text(ptext, wn_id, sname, ch_no, cksum, p_idx):
  mtl_data = MTL_TASK(ptext.split('\n'))
  print_msg("  - Đã phân tích xong phần %s, đang tải lên...", p_idx)

  json = {
    'wn_id': wn_id, 'sname': sname,
    'ch_no': ch_no, 'cksum': cksum,
    'p_idx': p_idx, '_algo': 'hmeg',
    'con_text': render_con_data(mtl_data["con"]),
    'mtl_json': mtl_data,
  }

  url = "https://chivi.app/_wn/anlzs/chaps"
  res = CLIENT.post(url, json = json)

  if res.status_code < 300:
    print_msg("  - Đăng tải thành công phần %s", p_idx)
  else:
    print(f"  - Đăng tải thất bại: {res.text}")


def analyze_book(wn_id, sname, chmin = 1, chmax = 0, force = False):
  out_dir = f"output/{wn_id}"
  os.makedirs(out_dir, exist_ok=True)

  for ch_no in range(chmin, chmax + 1):
    print(f"- Tải xuống text gốc chương [{ch_no}] nguồn [{sname}] truyện [{wn_id}]:")

    url = f"{URL}/texts/{wn_id}/{sname}/{ch_no}/parts"
    res = CLIENT.get(url).json()

    cksum = res["cksum"]
    parts = res["parts"]
    _hmeg = res["_hmeg"]

    print(f"  Đã lấy xuống text gốc, chương chia {len(parts) - 1} phần, checksum: [{cksum}].")
    print()

    for p_idx, ptext in enumerate(parts):
      if _hmeg[p_idx] == True and not force:
        print_msg("  - Phần %s đã có trên hệ thống, bỏ qua.")
        
      try:
        analyze_text(ptext, wn_id, ch_no, cksum, p_idx)
      except Exception as inst:
        print(inst)





## Bước 3: Đăng nhập vào hệ thống
Bạn hãy cung cấp địa chỉ email + mật khẩu tài khoản Chivi để hệ thống định danh


In [ ]:
do_login()

## Bước 4: Phân tích ngữ pháp cho bộ truyện

- ID bộ truyện: Con số đầu tiên trên địa chỉ bộ truyện. Ví dụ truyện có địa chỉ là https://chivi.app/wn/4-quy-bi-chi-chu/ thì ID bộ truyện là 4
- Nguồn chương: Tên danh sách chương, nguồn chính thức tên là ~chivi, nguồn tạm thời tên là ~draft, các nguồn cá nhân bắt đầu bằng @ tiếp đó là tên đăng nhập.
  

In [ ]:

print("Nhập ID bộ truyện:", end=' ')
wn_id = 1 # @params {type: "integer"}

print("Nhập nguồn chương:", end=' ')
sname = "~chivi" # params {type: "string"}

print("...Kiểm tra dữ liệu...", end=' ')

url = f"{URL}/seeds/{wn_id}/{sname}/brief"
res = requests.get(url).json()

total = res["chap_count"]
print(f"hoàn thành, nguồn truyện có tổng cộng {total} chương")

print("Bắt đầu từ chương (mặc định là 1): ", end=' ')
chmin = 1 # @params {type: "integer"}

print(f"Tới chương (mặc định là {total}): ", end=' ')
chmax = total # @params {type: "integer"}

if chmax < chmin:
  chmax = total

if chmin < 1:
  chmin = 1

print("Phân tích lại những chương đã có dữ liệu trên hệ thống: ")
force = False # #params {type: "boolean"}

print(f"Bạn sẽ chạy chương trình phân tích dữ liệu các chương từ {chmin} tới {chmax} của bộ truyện {wn_id}")
print(f"Lưu ý: Các chương đã được phân tích sẽ không phân tích lại, hãy kiểm tra trong thư mục output/{wn_id} nếu muốn dọn sạch rác!")

analyze_book(wn_id, sname, chmin, chmax, force)